In [59]:
import pandas as pd
import numpy as np
import glob
import os
import xml.etree.ElementTree as ET 
from datetime import datetime 
import requests
from bs4 import BeautifulSoup
import sqlite3

In [67]:

def extract(url , names):
    
    data_frame =pd.DataFrame( columns = names)


    response = requests.get(url).text
    soup = BeautifulSoup(response , "html.parser")
    table = soup.find_all("table" , {"class":"wikitable sortable mw-collapsible"})
    rows = table[0].find_all("tr")
    
    for row in rows:
        cols = row.find_all("td")
        if len(cols)!=0:
            dict_ = {
                
                "Name" : cols[2].text.strip(),
                "MC_USD_Billion" : cols[2].text.strip()
            }
            
            
            df = pd.DataFrame(dict_ , index = [0])

            data_frame = pd.concat([data_frame , df] , ignore_index=True)
    return(data_frame)

def transform(df , exchange_file):
    '''
    MC_GBP_Billion, GBP
    MC_EUR_Billion, EUR
    MC_INR_Billion, INR
    '''
    df_rate = pd.read_csv(exchange_file, index_col=[0])

    MC_USD_Billion_list = df["MC_USD_Billion"].tolist()
    MC_USD_Billion_list = [float(x) for x in MC_USD_Billion_list]
    df["MC_GBP_Billion"] = np.array(MC_USD_Billion_list)*df_rate.loc['GBP',:]["Rate"]
    df["MC_EUR_Billion"] = np.array(MC_USD_Billion_list)*df_rate.loc['EUR',:]["Rate"]
    df["MC_INR_Billion"] = np.array(MC_USD_Billion_list)*df_rate.loc['INR',:]["Rate"]
    return df

def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

def load_to_db(df, sql_connection, table_name):
  
    df.to_sql(table_name, sql_connection , if_exists = "replace" , index = False)
    


def run_query(query_statement, sql_connection):
    data = pd.read_sql(query_statement  ,sql_connection)
    
    return(data)
   
    

def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 
    
    
        

In [68]:
url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
names = ["Name" , "MC_USD_Billion"]
exchange_file = "exchange_rate.csv"
target = "Largest_banks_data.csv"
log_file = "code_log.txt"

log_progress("Preliminaries complete. Initiating ETL process")



extracted_data = extract(url , names)
log_progress("Data extraction complete. Initiating Transformation process")



transformed_data = transform(extracted_data , exchange_file)
log_progress("Data transformation complete. Initiating Loading process")



load_to_csv(transformed_data,target)
log_progress("Data saved to CSV file")


sql_connection = sqlite3.connect("Banks.db")
Table_name = "Largest_banks"
log_progress("SQL Connection initiated")


load_to_db(transformed_data ,sql_connection, Table_name)
log_progress("Data loaded to Database as a table, Executing queries")


query_statement = f"select * from {Table_name}"
run_query(query_statement, sql_connection)
log_progress("Process Complete")


sql_connection.close()
log_progress("Server Connection closed")







In [69]:
transformed_data

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,432.92,432.92,346.336,402.6156,35910.7140
1,231.52,231.52,185.216,215.3136,19204.5840
2,194.56,194.56,155.648,180.9408,16138.7520
3,160.68,160.68,128.544,149.4324,13328.4060
4,157.91,157.91,126.328,146.8563,13098.6345
5,155.87,155.87,124.696,144.9591,12929.4165
6,148.90,148.90,119.120,138.4770,12351.2550
7,140.83,140.83,112.664,130.9719,11681.8485
8,139.82,139.82,111.856,130.0326,11598.0690
9,136.81,136.81,109.448,127.2333,11348.3895
